# Forward Backward et FISTA.

##    NGUYEN Minh Hai - LE Cam Thanh Ha - 4 GMM A

Dans ce TP, nous allons utiliser l'algorithme dit Forward Backward et sa version accélérée appelée FISTA pour résoudre différents problèmes d'optimisation associés à de l'inpainting et du débruitage. Nous travaillerons sur les deux images de référence du TP précédent mais à nouveau, vous pouvez utiliser d'autres images.



In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
import requests
from PIL import Image
from io import BytesIO




In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
lenna = chargeData("Lenna")
cana = chargeData("Canaletto")
imagesRef= {"Lenna" : lenna,"Canaletto" : cana}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
def PSNR(I,Iref):
    temp=I.ravel()
    tempref=Iref.ravel()
    NbP=I.size
    EQM=np.sum((temp-tempref)**2)/NbP
    b=np.max(np.abs(tempref))**2
    return 10*np.log10(b/EQM)

# Inpainting

Pour le problème d'inpainting on va supposer qu'on dispose d'observations $y=Mx^0+b$ qui sont dégradées par un opérateur de masquage $M$ et éventuellement par un bruit additif $b$. On va chercher à estimer $x^0$ en minimisant une fonctionnelle de la forme 
\begin{equation}\label{Eq1}
\frac{1}{2}\Vert Mx-y\Vert^2+\lambda \Vert Tx\Vert_1
\end{equation}
où $T$ est une transformée dans laquelle on sait que l'image $x^0$ est parcimonieuse (typiquement une bonne base d'ondelettes comme db2). Pour utiliser le FB, il faut calculer le gradient du terme d'attache aux données et l'opérateur proximal de $\Vert Tx\Vert_1$. Vous aurez de l'aide plus loin dans le notebook pour cette partie.

Si le bruit est nul, c'est-à-dire si $y=Mx^0$, où $M$ est un opérateur de masquage, il serait même plus intéressant de déterminer une solution du problème d'optimisation suivant :
\begin{equation}\label{Eq2}
\underset{x}{\min}\Vert Tx\Vert_1\text{ sous la contrainte }y=Mx  
\end{equation}
Ce problème d'optimisation ne peut être résolu directement par par l'algorithme Forward-Backward (FB) car on ne peut pas le formuler comme une minimsaition d'une somme de deux fonctions dont l'unde est différentiable à gradient Lipschitz. 
De plus, en pratique, il est souvent illusoire de penser que les données $y$ ne sont pas corrompues par un bruit, aussi petit soit il. 

On peut démontrer que les solutions de \eqref{Eq1} convergent ver une solution de \eqref{Eq2} quand $\lambda$ tend vers 0. L'avantage de cette formulation \eqref{Eq1} est qu'elle peut être résolu par FB. 
Résoudre \eqref{Eq1} revient ainsi à la fois à inpainter et à régulariser. Plus $\lambda$ est grand plus on régularise. Dans la pratique, en l'absence de bruit, on a intérêt à choisir une petite valeur de $\lambda$ dans la formulation \eqref{Eq1}. Le problème dun tel choix est que ça ralentit l'algorithme : i.e il faut un plus grand nombre d'itérations pour atteindre un résultat comparable si on prend le même point de départ. Quand on utilise ces méthodes, les choix de $\lambda$, du nombre d'itérations et du point de départ de l'algorithme sont importants.

#### Commentaire :

<span style = "color:blue">
    
     On formalise notre problème de l'inpainting comme un problème d'optimisation, car en général on a un bruit additif $b$ et l'opérateur $M$ (qui est un projecteur orthogonale au point de vue Mathématique) n'est pas inversible. La norme $l_1$ nous aide à trouver une solution parcimonieuse. 
    
</span> 

M : projecteur orthogonale qui n'est pas inversible

norm l1 -> parcimoniouse

prox_g(x) = seuillage doux avec un seuil = gamma*lambda

# Rappels sur la descente de gradient explicite et sur Forward-Backward (FB)

Si $F=f+g$ est une fonction convexe composite, somme de deux fonctions convexes, $f$ différentiable à gradient 
$L$-Lipschitz et $g$ une fonction convexe dont on sait calculer l'opérateur proximal, l'algorithme Forward-Backward s'écrit 
$$x_{n+1}=prox_{hg}(x_n-h\nabla f(x_n))=Tx_n\quad \text{ avec }T:=prox_{hg}\circ (Id-h\nabla f)$$

On peut montrer que la suite de terme général $F(x_n)-F(x^*)$ est décroissante et de plus que 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn}$$
Cette vitesse en $\frac{1}{n}$ est optimale au sens où il n'est pas possible de trouver des bornes qui décroissent en $\frac{1}{n^{\delta}}$ avec $\delta>1$ pour toutes les fonctions convexes. Cela étant on peut montrer que si $h<\frac{1}{L}$ on a en fait 
$$F(x_n)-F(x^*)=o\left(\frac{1}{n}\right)$$
et que cette vitesse est même géométrique si la fonction $f$ est fortement convexe, dans le cas différentiable. 


## Remarques sur les algorithmes itératifs en général et FB en particulier.

Lorsqu'on utilise un algorithme itératif qui dépend d'un ou plusieurs paramètres, il faut être vigilent pour évaluer ses performances. Il est important de s'assurer d'une manière ou d'une autre des points suivants :

1) Vérifier qu'on est proche de la convergence... c'est-à-dire qu'on a fait suffisamment d'itérations. Ce nombre peut varier en fonction des paramètres de l'agorithme et du point initial $x_0$ choisi pour construire la suite.

Je vous invite pour cela à afficher la courbe de la valeur de la fonctionnelle et à prendre un peu de recul sur ce que vous observez. S'il reste des pixels noirs qui sont manifestement des résidus du masque ou si la fonctionnelle décroit encore fortement, c'est qu'il faut sans doute augmenter le nombre d'itérations. D'une manière générale, observer les artefacts (défauts) de l'image reconstruite peut donner des éléments de réponses.
De plus, observer la courbe de la valeur de la fonctionnelle peut parfois indiquer qu'on aurait pu effectuer moins d'itérations pour un résulat comparable. 

2) Explorer les valeurs des paramètres. Il se peut que certaines valeurs de paramètres fournissent des résultats pertinents, d'autres non. Avant de trancher sur le caractère efficace d'une méthode, il faut avoir pris le temps de vérifier que le choix des paramètres est correct. Les paramètres peuvent être ceux de la fonctionnelle à minimiser, comme le $\lambda$ de \eqref{Eq1} mais aussi des paramètres internes à l'algorithme comme le pas de descente par exemple. Le fait de faire un plan d'expériences dans ce cas sur des données tests peut être un bon moyen de déterminer des paramètres pertinents. 

On peut noter que la valeur de la fonctionnelle ne fait pas tout, en effet si on veut comparer différentes valeur de $\lambda$, c'est plutôt le PSNR de la reconstruction qui sera un critère pertinent. On ne peut pas se passer de réfléchir... 

3) Bien choisir la valeur initiale de la suite. Les algorithmes itératifs construisent de manière séquentielle des images qui s'approchent du minimiseur de la fonctionnelle. Le choix du point de départ de la suite peut être crucial.
Si on ne dispose d'aucune information a priori, on peut partir d'une image constante à 0 ou d'une image raisonnable.
Par exemple, de l'image masquée si on fait de l'inpainting, ou de l'image bruitée si on fait du débruitage.

On peut souvent distinguer deux phases dans l'optimisation de la fonctionnelle, une première qui permet de passer de l'image initiale vers une image raisonnable, une approximation grossière du résultat, puis une seconde qui permet de passer de cette première approximation grossière à une image très proche du résultat final souhaité.
Il est difficile d'esquiver la seconde phase mais on peut réduire drastiquement la première en proposant une image initiale déjà très raisonnable. On peut par exemple lancer un algorithme classique rapide, mais pas très précis pour donner une image initiale meilleure qu'une initialisation aléatoire ou naïve. 
Dans le cas du débruitage, on peut par exemple filtrer l'image originale ou la seuiller en ondelettes. Dans le cas de l'inpainting on peut appliquer localement un filtre médian. C'est-à-dire on découpe l'image en blocs de taille fixe (8 par 8 par exemple) et sur chaque bloc on estime la valeur médianne d'intensité lumineuse parmi les valeurs non masquées et on remplace, sur chaque bloc, les valeurs inconnues par cette valeur. C'est primitif mais rapide et ça permet d'avoir une première estimation raisonnable.

D'une manière générale, si vous voulez comparer plusieurs méthodes, avec des paramètres ou des bases d'ondelettes différentes, il est pertinent de calculer le PSNR des images reconstruites. 
C'est une manière d'obtenir un critère un minimum objectif.

## Mise en place de FB pour l'inpainting.

On pourra commencer dans un premier temps par des observations non bruitées et effectuer seulement un masquage. Pour créer un masque 2D, on peut utiliser les lignes de commandes suivantes :

In [ ]:
np.random.seed(seed = 1)
n1, n2 = np.shape(lenna)
r = np.random.rand(n1,n2)
M = (r < 0.5)
M = M * 1.0

In [ ]:
M



On peut bien entendu faire varier la proportion de l'image qu'on masque.

On masque l'image de la manière suivante :

In [ ]:
temp = M * lenna
pn.Row(hv.Image(lenna).opts(cmap = 'gray', width = 400, height = 400), hv.Image(temp).opts(cmap='gray', width=400, height=400))

<span style = "color:blue">
    
     En effectuant un masquage, il y a des pixels devenant noirs et l'image est bien masquée
    
</span> 

### Initilisation par un filtre médian.

Proposer et tester un algorithme de Filtre médian qui réalise un inpainting primaire en rempalçant les pixels manquants par la mediane des coefficients observés sur des carrés de taille vois*vois

In [ ]:
def FiltreMedian(im, masque, vois):
    imrec = np.copy(im)
    n1, n2 = np.shape(im)
    K1 = int(np.floor(n1/vois))
    K2 = int(np.floor(n2/vois))
    for k1 in np.arange(K1):   # Loop over dim = 0
        for k2 in np.arange(K2):  # Loop over dim = 1
            bloc = imrec[k1*vois:(k1+1)*vois, k2*vois:(k2+1)*vois]
            median = np.median(bloc[bloc != 0])
            imrec[k1*vois:(k1+1)*vois, k2*vois:(k2+1)*vois ] += (masque[k1*vois:(k1+1)*vois, k2*vois:(k2+1)*vois ] == 0)*median
            
    return(imrec)

#### Visualisation le résultat du filtre médian

In [ ]:
imrec2 = FiltreMedian(temp, M, 8)
pn.Row(hv.Raster(temp).opts(title = 'Image masquée', cmap='gray',xaxis=None,yaxis=None,width=350,height=350), 
          hv.Raster(imrec2).opts(title = 'Image filtre médian', cmap='gray',xaxis=None,yaxis=None,width=350,height=350))

In [ ]:
n1, n2 = np.shape(cana)
r = np.random.rand(n1,n2)
M_cana = (r < 0.5)
M_cana = M_cana * 1.0
cana_masque = M_cana * cana
imrec_cana = FiltreMedian(cana_masque, M_cana, 8)
pn.Row(hv.Raster(cana_masque).opts(title = 'Image masquée', cmap='gray',xaxis=None,yaxis=None,width=350,height=350), 
          hv.Raster(imrec_cana).opts(title = 'Image filtre médian', cmap='gray',xaxis=None,yaxis=None,width=350,height=350))

#### Commentaire
<span style = "color:blue">
    
     Le filtre médian est simple à implémenter et rapide à exécuter. Pourtant, il nous donne une très bonne initialisation pour notre algorithme d'optimisation itérative. 
Cette étape est très important en optimisation, même on a un problème d'optimisation convexe et l'algorithme Forward-Backward assure une convergence globale, un point 
de départ qui n'est pas très loin de la solution optimale nous aide à réduire le nombre d'itérations donc le temps d'éxecution. 
    
</span> 



### Retour sur l'algorithme FB

Proposer une fonction qui calcule le gradient de la fonction $f(x)=\frac{1}{2}\Vert Mx-y\Vert^2$ :

<span style = "color:blue">

Car M est un projecteur orthogonal donc $M^T = M$ et $M \circ M = M$
    
On en obtient le gradient : $$ \Delta f(x) = M^T(Mx-y) = M^T M x -M^T y = M(x-y)$$
    
    
</span> 

In [ ]:
def GradientInpainting(x, y, M):
    g = M*(x - y)
    return g

Pour quelle valeur $L$ ce gradient est-il $L$-Lipschitz ?

<span style = "color:blue">
 On a:

\begin{equation}
    \lVert \nabla f(x)-\nabla f(y) \rVert = \lVert M(x-y) \rVert \leq \lVert M \rVert \times \lVert x - y \rVert
\end{equation}

Donc la fonction $f$ est à gradient $L$-lipschitz avec $L$ = $ \lVert M \rVert$.

De plus, comme $M$ est un projecteur orthogonale, il est de norme $1$, donc $f$ est à gradient $1$-lipschitz
</span> 


Les fonctions suivantes permettent d'évaluer l'opérateur proximal de la fonction 
$g(x)=\lambda \Vert Tx\Vert_1$ où $T$ est une transformée orthogonale en ondelettes ou une transformée en ondelettes à trou, c'est-à-dire invariante par translation. Vous noterez qu'une transformée en Ondelettes de Daubechies db2 à été choisie ici par défaut mais que ce ci peut bien entendu être changé.

In [ ]:
def SeuillageDouxOndelettes(I, wave, Seuil):
    '''
    i.e l'opérateur proximal de lambda * norme_l1(Tx) , avec lambda = Seuil
    '''
    L = pywt.dwt_max_level(len(I), pywt.Wavelet(wave).dec_len)
    wavelet_coeffs = pywt.wavedecn(I, wave, mode = 'per', level = L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp = pywt.threshold(arr, Seuil, mode = 'soft')
    test = pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format = 'wavedecn')
    Irec = pywt.waverecn(test, wave, mode = 'per')
    return Irec

def Normel1Ondelettes(I, wave):
    L = pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs = pywt.wavedecn(I, wave, mode = 'per', level = L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1 = sum(np.abs(arr))
    return norml1

In [ ]:
wave = 'haar'
imrec = SeuillageDouxOndelettes(lenna, wave, 3)

Proposer et tester un code d'inpaiting utilisant Forward-Backward minimisant la fonctionnelle (1) et la tester.

On pensera à cliper le résultat entre 0 et 255.

In [ ]:
def Loss_F(z, y, M, lamb, wave):
    return 0.5 * np.sum((np.abs(M*z - y)**2)) + lamb * Normel1Ondelettes(z, wave)
    

In [ ]:
def ForwardBackwardInpaintingv2(x, y, M, step, lam, Niter, wave):
    z = FiltreMedian(y, M, 8)
    PSNR_list = [PSNR(z, x)]
    Losses = [Loss_F(z, y, M, lam, wave)]
    
    for i in range(Niter):
        grad = GradientInpainting(z, y, M)   # gradient de f(s)
        z = SeuillageDouxOndelettes(I = z - step*grad, wave = wave, Seuil = step*lam)        # Calculer l'operateur proximal
        PSNR_list.append(PSNR(z, x))
        Losses.append(Loss_F(z, y, M, lam, wave))
    
    PSNR_list = np.array(PSNR_list)
    Losses = np.array(Losses)
    
    return np.clip(z, 0, 255), PSNR_list, Losses

### Appliquer à l'image masquée de Lenna

In [ ]:
y = M * lenna
step = 0.95
lam = 5
Niter = 100
wave = 'haar'
Ta = 350
imrec, psnr, losses = ForwardBackwardInpaintingv2(lenna, y, M, step, lam, Niter, wave)
pn.GridBox(hv.Image(lenna).opts(title = "Image original", cmap = 'gray', width = Ta, height = Ta),
       hv.Image(y).opts(title = "Image masquée", cmap = 'gray', width = Ta, height = Ta),
       hv.Image(FiltreMedian(y, M, 8)).opts(title = "Filtre médian", cmap = 'gray', width = Ta, height = Ta)
       ,hv.Image(imrec).opts(title = "Image reconstruite par FB", cmap='gray', width = Ta, height = Ta), ncols = 2)

In [ ]:
hv.Curve(psnr).opts(title = "Evolution de PSNR", width = 600, height = 300)

In [ ]:
hv.Curve(losses).opts(title = "Evolution de la fontion coût", width = 600, height = 300)

#### Commentaire
<span style = "color:blue">
    
     En minimisant la fonction coût et utilisant une transformée en endelettes de Haar, on a obtenu un résultat assez proche de l'image initiale. Grâce à l'initialisation par filtre médian, l'algorithme converge rapidement (avec 100 itérations on obtient une image assez claire). Par contre, on peut constater que la solution obtenue contient des effets "carrés", c'est normal parce qu'on a utilise les ondelettes de Haar dont les fonctions d'échelles sont constantes par morceaux. De plus, la fonction coût et le PSNR ne varie pas beaucoup a partir de 80e itération. 
</span> 


### Dashboard

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
imagesRef= {"Lenna" : lenna,"Canaletto" : cana}

Proposer un dashboard permettant une exploration numérique de la fonction précédente. On pourra ainsi observer que les petites valeurs de $\lambda$ donnent de meilleurs résulats. En théorie, si les données observées ne sont pas bruitées, il faudrait prendre une valeur de $\lambda$ aussi petite que possible. Vous pouvez le remarquer si vous affichez le PSNR de l'image reconstruite et que vous faites un nombre suffisant d'itérations.

Le problème d'un tel choix est que cela ralentit considérablement l'algorithme. Si les données sont bruitées, la valeur optimale de $\lambda$ est proportionnelle à lécart type du bruit.  

Ainsi, vous aurez intérêt à choisir une valeur faible de $\lambda$ mais pas trop petite pour assurer que votre algorithme est proche de la convergence.

In [ ]:
class FBInpaint(param.Parameterized):
    wave = param.ObjectSelector(default = "haar", objects = wavelist)
    image = param.ObjectSelector(default = "Canaletto", objects = imagesRef.keys())
    Niter = param.Integer(10, bounds = (0,300))
    lam = param.Number(10, bounds = (1,30))
    step = param.Number(0.9, bounds = (0.5,4))
    masquage = param.Number(0.5, bounds = (0.1,1))
    def view(self):
        I = imagesRef[self.image]
        n1, n2 = np.shape(I)
        r = np.random.rand(n1, n2)
        M = (r < self.masquage)
        M = M * 1.0
        im_mas = M * I
        im_rec, psnr, losses = ForwardBackwardInpaintingv2(x = I, y = im_mas, M = M, step = self.step, lam = self.lam, Niter = self.Niter, wave = self.wave)
        
        return pn.GridBox(hv.Image(I).opts(title = "Image original", cmap = 'gray', width = 350, height = 350),
                     hv.Image(im_mas).opts(title = "Image masquée", cmap = 'gray', width = 350, height = 350),
                     hv.Image(im_rec).opts(title = "Image reconstruite", cmap='gray', width = 350, height = 350), ncols = 2)
        

In [ ]:
fbinpaint = FBInpaint()
pn.Row(fbinpaint.param, fbinpaint.view)

<span style = 'color:blue'>

#### Commentaire

- Globalement, cette modélisation du problème de l'inpaiting sous forme d'un problème d'optimisation convexe utilisant les bases d'ondelettes est très efficace, car on a des bonnes propriétés théoriques ainsi que l'algorithme Forward-Backward donne les bons résultats
- Bases ondelettes: la base de Haar a une présence des effets carrés car les fonctions sont constantes par morceaux.
- L'effet du bord: on constate des pertes d'informations sur les bords de l'image, car on a calculé la transformée en ondelette avec le mode 'périodique'
- Le paramètre `lambda`: mesure l'effet de la transformée en ondelettes, plus `lambda` est grand, plus on force la transformée en ondelette $Tx$ être parcimonieuse, donc plus de propriétés des ondelettes influencent le résultat. On peut voir très  clair dans le cas de Haar, si `lambda` est grand, l'effet de carré est plus nette.
- Le nombre d'itération: grâce à l'initialisation par le filtre médian, on a réduit significativement le nombre d'itérations.
- Le pas de descent: théoriquement, l'algorithme Forward-Backward est un algorithme de descent si le pas $s < \frac{2}{L} = 2$. Donc on peut accélérer la convergence en mettant le pas $ s = 1.99$. En pratique, on peut même mettre le pas $s$ plus grand que $2$ pour accélérer la convergence, mais il faut faire attention car si $s > 2$, on n'est pas assuré la convergence.

# Accélération de Nesterov, FISTA.

Yuri Nesterov a proposé dans les années 1980 plusieurs méthodes permettant l'accélération de la descente de gradient explicite. Nous allons nous intéresser à une en particulier, celle qui est détaillée dans le polycopié de cours. Si vous faites des recherches bibliographiques, soyez conscients que sous le terme "Accélération de Nesterov" peuvent se cacher différentes méthodes, par interpolation ou extrapolation, spécifiques aux fonctions fortement convexes ou pas, s'appliquant à une descente de gradient simple ou à des fonctions composites. De plus nous n'utiliserons pas les paramètres historiques proposés par Nesterov, mais ceux qu'Antonin Chambolle et moi même avons proposé en 2014 pour en améliorer la vitesse et la convergence.

L'accélération de la descente de gradient proposée Yurii Nesterov en 1984 et adapatée à FB sous le nom de FISTA par Beck et Teboulle en 2009 est d'une mise en oeuvre très simple.

Il suffit d'effectuer la descente de gradient en un point décalé de $x_n$ avec un pas $h<\frac{1}{L}$ (attention cette borne est plus contraignante que pour la descente de gradient classique) :
$$x_{n+1}=T(x_n+\alpha_n(x_n-x_{n-1}))$$ 
où la suite $\alpha_n$ est bien choisie et $T$ st soit l'opérateur de descente de gradient explicite, soit l'opérateur associé au FB. On parle de méthode inertielle car cette méthode utilise un terme dit de "mémoire" ou inertiel qui exploite la dernière direction de descente.

Donnons quelques éléments clés importants :

1) Le choix original de Nesterov pour la suite $\alpha_n$ ets le suivant :
\begin{equation}
\alpha_n=\frac{t_n-1}{t_{n+1}}\text{ avec }t_1=1\text{ et }t_{n+1}=\frac{1+\sqrt{1+t_n^2}}{2}
\end{equation}
2) Pour ce choix on a 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn^2}$$
3) On peut prendre plus simplement $\alpha_n=\frac{n-1}{n+a-1}$ avec $a>3$ (c'est ce que vous programmerez) dans ce cas, on a $$F(x_n)-F(x^*)\leqslant \frac{(a-1)^2\Vert x_0-x^*\Vert^2}{2h(n+a)^2}$$ et en plus $F(x_n)-F(x^*)=o\left(\frac{1}{n^2}\right)$ et on a convergence de la suite $(x_n)_{n\geqslant 1}$. On peut noter que dans ce cas, la première étape est une simple sans inertie ($\alpha_1=0$) et donc $x_1=T(x_0)$. L'inertie apparait pour le calcul de $x_2$. Le choix originel de Nesterov est très proche du choix $a=3$. 

4) Dans le cas d'un fonction composite, on parle de l'algotithme FISTA (Fast Iterative Soft Shrinckage Algorithm) car si $g$ est une norme $\ell_1$, l'opérateur proximal se réduit à un seuillage doux. Mais FISTA est le nom générique proposé par Beck et Teboulle en 2009, ce peut être trompeur.

5) La suite de terme général $F(x_n)-F(x^*)$ n'est pas nécessairemment décroissante comme dans le cas de FB ou de la descente de gradient. Les bornes données précédemment sont des bornes mais ne reflètent pas la décroissance réelle. Dans la pratique vous verrez que FISTA est quand même plus rapide que FB.


Proposer un algorithme d'inpainting consistant à minimiser la fonctionnelle (1) en utilisant FISTA. 

In [ ]:
def FISTAInpainting(x, y, M, step, lam, Niter, wave, alpha):
    z = FiltreMedian(y, M, 8)
    f = [PSNR(z, x)]
    
    # First iteration alpha = 0
    grad = GradientInpainting(z, y, M)
    z_old = z
    z = SeuillageDouxOndelettes(z - step*grad, wave, Seuil = step*lam)
    f.append(PSNR(z, x))
    
    for k in range(1, Niter):
        temp = z + (k/(k + alpha))*(z - z_old)
        grad = GradientInpainting(temp, y, M)
        z_old = z
        z = SeuillageDouxOndelettes(temp - step*grad, wave, Seuil = step*lam)
        f.append(PSNR(z, x))
        
    f = np.array(f)
    
    return np.clip(z, 0, 255), f

In [ ]:
imrec, f = FISTAInpainting(lenna, y, M, step, lam, 75, wave, 4)
pn.Row(hv.Image(lenna).opts(title = 'Image original', cmap = 'gray', width = Ta, height = Ta),
       hv.Image(y).opts(title = 'Image masquée', cmap = 'gray', width = Ta, height = Ta),
       hv.Image(imrec).opts(title = 'Image reconstruite par FISTA', cmap = 'gray', width = Ta, height = Ta))

In [ ]:
hv.Curve(f).opts(title = "Evolution de PSNR", xlabel = 'Itération', ylabel = 'PSNR', width = 600, height = 300)

### Dashboard

In [ ]:
class FISTAInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(10,bounds=(0,300))
    lam = param.Number(10,bounds=(1,30))
    step = param.Number(0.9,bounds=(0.5,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    alpha = param.Number(3,bounds=(1,10))
    def view(self):
        I = imagesRef[self.image]
        n1, n2 = np.shape(I)
        r = np.random.rand(n1, n2)
        M = (r < self.masquage)
        M = M * 1.0
        im_mas = M * I
        im_rec, psnr = FISTAInpainting(x = I, y = im_mas, M = M, step = self.step, lam = self.lam, Niter = self.Niter, wave = self.wave, alpha = self.alpha)
        
        return pn.GridBox(hv.Image(I).opts(title = "Image original", cmap = 'gray', width = 350, height = 350),
                     hv.Image(im_mas).opts(title = "Image masquée", cmap = 'gray', width = 350, height = 350),
                     hv.Image(im_rec).opts(title = "Image reconstruite par FISTA", cmap='gray', width = 350, height = 350), ncols = 2)
        

In [ ]:
fistainpaint= FISTAInpaint()
pn.Row(fistainpaint.param, fistainpaint.view)

<span style = 'color:blue'>

#### Commentaire

- Cette méthode FISTA converge plus rapide que Forward-Backward, elle a besoin moind d'itérations pour converger 
- Les mêmes effets carrés avec les ondelettes de Haar
- FISTA est moins sensible avec le paramètre `lambda`, elle peut converger vers une bonne solutions avec différentes valeurs de `lambda`
- Il y a toujours les effets du bord de l'image (à cause de la transformée périodique)
- Attention: il ne faut pas mettre un pas trop grand et beaucoup d'itération car un pas trop grand ne assure pas la convergence et peut éventuellement donner le mauvais résultat. Mais un pas de descent raisonnable peut accélérer encore la convergence

## Pour aller plus loin (facultatif et à titre d'information) - Inpainting en utilisant les bases d'ondelettes translattées (décalage)

Comme pour le débruitage, on peut utiliser des bases d'ondelettes translatées pour améliorer la qualité de l'inpainting. Dans ce cas, il ne faut évidemment pas recalculer l'inpainting depuis le début pour chaque version translatée, mais utiliser comme initialisation les versions inpaintées précédentes. 
Cette variante permet également de lisser les défauts et supprime des effets liés à la forme de l'ondelettes et donc les effets de blocs si on utilise les ondelettes de Haar.

#### Principle
Dans cette partie nous allons améliorer les résultats obtenus précédamment en combinant plusieurs solutions décalées dans les bases d'ondelettes.
L'idée de cette méthode est:
- Decaler l'image i pixels horizontalement et j pixel verticalement avec $(i,j)\in$ {1,2,3,4}$^2$
- Pour chaque version decalée, effectuer FISA et decaler dans le sens inverse resulat obtenu.
- Calculer la moyenne. On obtient le résultat final.

#### On ecrit d'abord une fonction FISTA qui permet de résoudre le problème avec un point de départ précis

In [ ]:
def FISTAInpaintingInit(x, y, M, step, lam, Niter, wave, alpha, z_init):
    z = z_init
    
    # First iteration alpha = 0
    grad = GradientInpainting(z, y, M)
    z_old = z
    z = SeuillageDouxOndelettes(z - step*grad, wave, Seuil = step*lam)
    
    for k in range(1, Niter):
        temp = z + (k/(k + alpha))*(z - z_old)
        grad = GradientInpainting(temp, y, M)
        z_old = z
        z = SeuillageDouxOndelettes(temp - step*grad, wave, Seuil = step*lam)

    return np.clip(z, 0, 255)

### FISTA avec translation

In [ ]:
def FISTAInpaintingTranslation(x, y, M, step, lam, Niter, wave, alpha, NbT):
    z, _ = FISTAInpainting(x, y, M, step, lam, Niter, wave, alpha)
    sum = np.copy(z)
    
    for k in range(NbT):
        z_h = np.roll(z, k, axis = 0)
        x_h = FISTAInpaintingInit(x, y, M, step, lam, Niter, wave, alpha, z_init = z_h)
        z_h = np.roll(x_h, -k, axis = 0)
        
        z_h_2 = np.roll(z, -k, axis = 0)
        x_h_2 = FISTAInpaintingInit(x, y, M, step, lam, Niter, wave, alpha, z_init = z_h_2)
        z_h_2 = np.roll(x_h_2, k, axis = 0)
        
        z_v = np.roll(z, k, axis = 0)
        x_v = FISTAInpaintingInit(x, y, M, step, lam, Niter, wave, alpha, z_init = z_v)
        z_v = np.roll(x_v, -k, axis = 0)
        
        z_v_2 = np.roll(z, -k, axis = 0)
        x_v_2 = FISTAInpaintingInit(x, y, M, step, lam, Niter, wave, alpha, z_init = z_v_2)
        z_v_2 = np.roll(x_v_2, k, axis = 0)
        
        sum_k = z_h + z_h_2 + z_v + z_v_2
#         sum_k = x_h + x_h_2 + x_v + x_v_2
        sum = sum + sum_k
        z = sum_k/4
        
    return sum / (NbT*4 + 1)

In [ ]:
y = M * lenna
step = 0.95
lam = 5
Niter = 100
wave = 'haar'
Ta = 350
imrec, _ = FISTAInpainting(lenna, y, M, step, lam, 150, wave, 3)
imrec_trans = FISTAInpaintingTranslation(lenna, y, M, step, lam, 150, wave, 3, NbT = 3)

print('PSNR de FISTA : ', PSNR(imrec, lenna))
print('PSNR de FISTA avec translation : ', PSNR(imrec_trans, lenna))

pn.GridBox(hv.Image(lenna).opts(title = "Image original", cmap = 'gray', width = Ta, height = Ta),
       hv.Image(y).opts(title = "Image masquée", cmap = 'gray', width = Ta, height = Ta),
       hv.Image(imrec).opts(title = "Inpaiting par FISTA", cmap = 'gray', width = Ta, height = Ta),
       hv.Image(imrec_trans).opts(title = "Inpaiting par FISTA et translations", cmap = 'gray', width = Ta, height = Ta), ncols = 2 )


<span style = 'color:blue'>

### Commentaires:
    
En utilisant les translations, on a gagné un gain en terme de PSNR et le résultat est plus 'lisse', il y a moins de l'effet carré

## Débruitage par minimisation de la variation totale

Si l'image qu'on souhaite débruiter une image dont on sait qu'elle a une faible variation totale (norme $\ell_1$ du gradient), on peut estimer l'image en minimisant une fonctionnelle . Attention, on parle ici du gradient de l'image, pas d'une fonctionnelle qui associe une image à un réel. 
Le gradient d'une image est un champ de vecteur avec deux composantes, l'une verticale, l'autre horizontal qui se calcule de manière classique par différence finie. J'ai fait le choix dans les codes proposés d'une discrétisation du gradient et de la discrétisation de l'opérateur adjoint associé. En particulier, cette version n'est pas périodisée. Si vous voulez utiliser une autre implémentation du gradient discret, il est probable qu'il faille recoder l'opérateur 
adjoint.

Si on note $y=x^0+b$ les observations où $x^0$ est l'image à estimer et $b$ un bruit additif, cette fonctionnelle sera de la forme 
\begin{equation}\label{Primal}\tag{Primal}
\frac{1}{2}\Vert x-y\Vert^2+\lambda \Vert \nabla x\Vert_1
\end{equation}

Minimiser une telle fonctionnelle revient à caluler l'opérateur proximal de la fonction 
$g(x)=\lambda \Vert \nabla x\Vert_1$, or il n'existe pas de formule pour un tel opérateur. Ce qui implique aussi que réaliser un Forward Backward avec le terme quadratique comme terme différentiable ne peut pas fonctionner, il nécessite la connaissance de cet opérateur proximal.

Il existe cependant un moyen d'utiliser l'algorithme FB pour résoudre ce problème : on procède par dualisation.
Il n'est pas possible de détailler cette approche ici, je vous renvoie sur le Poly. La dualisation qui utilise la conjugué de Fenchel Rockafellar permet de montrer qu'on peut associer au problème précédent, le problème doptimisation suivant
\begin{equation}\label{Dual}\tag{Dual}
\min_{p}\frac{1}{2}\Vert div(p)+y\Vert^2+\iota_{\mathcal{B}_{\infty,\lambda}}(p)
\end{equation}
où $p$ est un champ de gradient et 
où $\iota_{\mathcal{B}_{\infty,\lambda}}$ est la boule $\ell_{\infty}$ de rayon $\lambda$ et où $div$ est l'opérateur 
de divergence. La présence de la divergence est ici due au fait que $-div$ est l'opérateur adjoint du gradient $\nabla$. La norme $\infty$ est ici présente car c'est la norme duale de la norme $/ell_1$.

Si $p^*$ est la solution du problème précédent alors $x^*=y+div(p^*)$ est la solution du problème initial.

L'avantage de ce second problème est qu'il peut être résolu par l'algorithme FB dans la mesure où le terme non différentiable est une indicatrice d'un ensemble sur lequel on sait projeter.

Les fonctions suivantes permettent de calculer le gradient et la divergence.
Tel que tout est codé ici, il s'agit d'une divergence négative, c'est à dire l'opposé de la divergence. A vous de voir si vous voulez tout remodifier. 

In [ ]:
def GradientHor(x):
    y = x - np.roll(x, 1, axis=1)
    y[:,0] = 0
    return y

def GradientVer(x):
    y = x - np.roll(x, 1 , axis=0)
    y[0,:] = 0
    return y

def DivHor(x):
    N = x.shape[1]
    y = x - np.roll(x, -1, axis=1)
    y[:, 0] = -x[:,1]
    y[:, N-1] = x[:, N-1]
    return y

def DivVer(x):
    N = x.shape[0]
    y = x - np.roll(x, -1, axis=0)
    y[0,:] = -x[1, :]
    y[N-1,:] = x[N-1,:]
    return y

def Gradient(x):
    y = []
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return np.array(y)

def Div(y):
    # -div
    x = DivHor(y[0]) + DivVer(y[1])
    return x

In [ ]:
def ProjGradBouleInf(g, lamb):
    # L'opérateur proximal
    gh = g[0]
    gv = g[1]
    temp = np.copy(g)
    p0 = gh - (gh - lamb)*(gh > lamb) - (gh + lamb)*(gh < -lamb)
    p1 = gv - (gv - lamb)*(gv > lamb) - (gv + lamb)*(gv < -lamb)
    temp[0] = p0
    temp[1] = p1
    return temp

### Fonction Python qui résout le problème dual par Forward Backward

In [ ]:
def FBDenoisingTV(y, lamb, step, Niter):
    
    p = np.array([np.zeros(y.shape), np.zeros(y.shape)])
    
    for k in range(Niter):
        grad = Gradient(- Div(p)) 
        # grad = np.dot(grad, - Div(p) + y)
        grad[0] = np.dot(grad[0], - Div(p) + y)
        grad[1] = np.dot(grad[1], - Div(p) + y)
        
        p = ProjGradBouleInf(p - step * grad, lamb)
    
    x  = y + Div(p)
    return np.clip(x, 0, 255)

Tester ce code sur un exemple d'image bruitée par un bruit gaussien. L'image résultante subit un effet Cartoon. Pourquoi ? Faite varier les paramètres pour voir les effets des différents paramètres de l'algorithme.  
Le pas doit être maintenu en dessous de $1/8$ pour vérifier les conditions de Lipschitz du gradient.

### Testons Forward-Backward pour minimiser la variation totale avec Lenna

In [ ]:
n1, n2 = np.shape(lenna)
Sigma = 30
Noise = np.random.rand(n1, n2)
lenna_noised = lenna + Sigma * Noise

lenna_denoised = FBDenoisingTV(lenna_noised, lamb = 5, step = 0.1, Niter = 1000)

print(PSNR(lenna_denoised, lenna))

pn.Row(hv.Image(lenna).opts(title = 'Image originale', cmap='gray',width = 350, height = 350),\
       hv.Image(np.clip(lenna_noised, 0, 255)).opts(title = 'Image bruitée',cmap = 'gray',width = 350, height = 350)\
       ,hv.Image(lenna_denoised).opts(title = 'Image débruitée par Variation Totale',cmap = 'gray', width = 350, height = 350))



<span style = 'color:blue'>

### Commentaires:
    
On a obtenu un résultat assez clair sans perte d'informations et on peut éviter les effets de bords qu'on a vu dans le cas d'ondelettes

Créer le dashboard associé et tester l'algorithme sur différentes images et faites varier les paramètres.
Le dashboard doit renvoyer 3 images et 2 PSNR. Vous devrier observer un effet "cartoon" surtout pour de grandes valeurs du paramètre $\lambda$. 

In [ ]:
class DenoisingTVFB(param.Parameterized):
    Niter = param.Integer(100,bounds=(100,1000))
    image = param.ObjectSelector(default="Lenna", objects=imagesRef.keys())
    lamb = param.Number(9,bounds=(0,50))
    step = param.Number(0.1,bounds=(0.1,2))
    Sigma = param.Number(30,bounds=(1,100))
    
    def view(self):
        I = imagesRef[self.image]
        n1, n2 = np.shape(I)
        Noise = np.random.rand(n1, n2)
        IB = I + self.Sigma * Noise
        IDb = FBDenoisingTV(IB, self.lamb, self.step, self.Niter)
        print('PSNR image bruitée: ', PSNR(IB, I))
        print('PSNR image débruitée: ', PSNR(IDb, I))
        
        return pn.Row(hv.Image(np.clip(IB, 0, 255)).opts(title = 'Image bruitée',cmap = 'gray',width = 350, height = 350)\
                   ,hv.Image(IDb).opts(title = 'Image débruitée par Variation Totale',cmap = 'gray', width = 350, height = 350))
        
        
    

In [ ]:
denoisingTVFB = DenoisingTVFB()
pn.Row(denoisingTVFB.param, denoisingTVFB.view)

<span style = 'color:blue'>

### Commentaires:
- Le paramètre `lambda` est très important, si il est grand, on rencontre l'effet de 'cartoon', et c'est difficile pour trouver le `lambda` optimal
- 

### Minimisation la Variation Totale par FISTA

In [ ]:
def FISTADenoisingTV(y, lamb, step, Niter, alpha = 3):
    
    p = np.array([np.zeros(y.shape), np.zeros(y.shape)])

    grad = Gradient(- Div(p)) * (- Div(p) + y)
    # grad[0] = np.dot(grad[0], - Div(p) + y)
    # grad[1] = np.dot(grad[1], - Div(p) + y)
    
    p_old = p
    p = ProjGradBouleInf(p - step * grad, lamb)
    
    for k in range(Niter):
        temp = p + (k / (k + alpha)) * (p - p_old)
        grad = Gradient(- Div(temp)) * (- Div(temp) + y)
        # grad = np.dot(grad, - Div(p) + y)
        # grad[0] = np.dot(grad[0], - Div(temp) + y)
        # grad[1] = np.dot(grad[1], - Div(temp) + y)
        p_old = p        
        p = ProjGradBouleInf(temp - step * grad, lamb)
    
    x  = y + Div(p)
    return np.clip(x, 0, 255)

In [ ]:
n1, n2 = np.shape(lenna)
Sigma = 30
Noise = np.random.rand(n1, n2)
lenna_noised = lenna + Sigma * Noise

lenna_denoised_FISTA = FISTADenoisingTV(lenna_noised, lamb = 3.1, step = 0.1, Niter = 10000, alpha = 3)

print(PSNR(lenna_noised, lenna))
print(PSNR(lenna_denoised_FISTA, lenna))

pn.Row(hv.Image(lenna).opts(title = 'Image originale', cmap='gray',width = 350, height = 350),\
       hv.Image(np.clip(lenna_noised, 0, 255)).opts(title = 'Image bruitée',cmap = 'gray',width = 350, height = 350)\
       ,hv.Image(lenna_denoised_FISTA).opts(title = 'Image débruitée par Variation Totale - FISTA',cmap = 'gray', width = 350, height = 350))



## A retenir.

Si vous avez une descente de gradient ou un Forward-Backward à implémenter, les accélérations de Nesterov sont très souvent efficaces et il est toujours rentable de tester et même de tester différentes valeurs de $a$. Il est à noter toutefois que théoriquement sur des fonctions fortement convexes, pour l'algorithme FB, la suite de terme général $F(x_n)-F(x^*)$ décroit au moins géométriquement ce qui n'est pas le cas de l'accélération de Nesterov présentée ici qui sont au mieux polynomiales.
Dans la pratique, pour un nombre raisonnable d'itérations, même dans ce cas, les accélérations de Nesterov sont largement compétitives.
Mais il en existe d'autres encore plus efficaces spécifiquement dédiés à ce cas.